In [ ]:
%pip install llama-index-embeddings-huggingface

In [ ]:
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

# 환경 변수 설정
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Ollama LLM 설정
llm = Ollama(
    model="deepseek-r1:8b",  # Ollama에서 제공하는 deepseek-r1:8b 모델 사용
    temperature=0.1,  # 응답의 창의성 조절 (0에 가까울수록 일관된 응답)
    context_window=4096,  # 컨텍스트 윈도우 크기
    timeout=120,  # 타임아웃 시간을 120초로 설정
    request_timeout=120,  # 요청 타임아웃도 120초로 설정
    additional_kwargs={  # 추가 파라미터
        "num_gpu": 0,  # CPU 사용 설정
        "seed": 42,  # 재현성을 위한 시드 설정
    },
)

# BGE 임베딩 모델 설정
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-m3", device="cpu"  # CPU 사용. GPU의 경우 "cuda"
)

In [ ]:
# ServiceContext 설정
Settings.llm = llm
Settings.embed_model = embed_model

# PDF 문서 로드
documents = SimpleDirectoryReader(
    input_files=["../data/pdf_sample1/240828_(AI리포트)_미국의_인공지능(AI)_정책,전략.pdf"],
    filename_as_id=True).load_data()

In [ ]:
# 인덱스 생성
index = VectorStoreIndex.from_documents(documents)

# 쿼리 엔진 생성
query_engine = index.as_query_engine(
    streaming=True,  # 스트리밍 응답 활성화
    similarity_top_k=3,  # 상위 3개의 가장 관련성 높은 문서 청크 사용
)

In [ ]:
# 질문하기
response = query_engine.query(
    """
    책임 있는 AI 개발에서 미국의 리더십을 명확히 하는 데에
    AI안전 연구소가 맡은 중요한 역할이 있다고 생각한다고 말한 사람과 
    그 사람이 누구와 협력한다고 했는지 알려줘
    """
)
print(response)